# Jupyter Example
## LAMMPS: body/in.cubes
Reference: https://github.com/lammps/lammps/blob/master/examples/body/in.cubes  
Required files: data.cubes  
Description: 3d rounded cubes

In [1]:
from __future__ import print_function
import random, math
from lammps import IPyLammps

In [2]:
L = IPyLammps()
L.variable("r index 3")
L.variable("steps index 10000")
L.units("lj")
L.dimension("3")
L.atom_style("body rounded/polyhedron 1 10")
L.read_data("data.cubes")
L.replicate("$r $r $r")
L.velocity("all create 1.2 187287 dist gaussian mom yes rot yes")
L.variable("cut_inner equal 0.5")
L.variable("k_n equal 100")
L.variable("k_na equal 1")
L.variable("c_n equal 20")
L.variable("c_t equal 5")
L.variable("mu equal 0")
L.variable("A_ua equal 1")
L.pair_style("body/rounded/polyhedron ${c_n} ${c_t} ${mu} ${A_ua} ${cut_inner}")
L.pair_coeff("* * ${k_n} ${k_na}")
L.comm_modify("vel yes")

LAMMPS output is captured by PyLammps wrapper


In [3]:
L.neighbor("0.5 bin")
L.neigh_modify("every 1 delay 0 check yes")

#L.fix("1 all nve/body")
L.fix("1 all nvt/body temp 1.2 1.2 0.1")
#L.fix("1 all npt/body temp 1.2 1.2 0.1 iso 0.002 0.02 1.0")

In [4]:
L.compute("p2 all pressure 1_temp")
L.compute("1 all body/local id 1 2 3")

In [5]:
L.dump("1 all local 1000 dump.* index c_1[1] c_1[2] c_1[3] c_1[4]")

#L.dump("2 all image 1000 image.*.jpg type type zoom 1.5 adiam 1.5 body type 0 0 view 60 15")
#L.dump_modify("2 pad 6")

L.dump("2 all movie 1000 mov.mp4 type type zoom 1 adiam 1.5 body type 0 0 view 60 15")
L.dump_modify("2 pad 6")

In [6]:
L.thermo_style("custom step ke pe etotal c_p2 c_1_temp")
L.thermo("1000")
L.timestep("0.001")
L.run("${steps}")

['Neighbor list info ...',
 '  update every 1 steps, delay 0 steps, check yes',
 '  max neighbors/atom: 2000, page size: 100000',
 '  master list distance cutoff = 3.9641',
 '  ghost atom cutoff = 3.9641',
 '  binsize = 1.98205, bins = 10 10 10',
 '  1 neighbor lists, perpetual/occasional/extra = 1 0 0',
 '  (1) pair body/rounded/polyhedron, perpetual',
 '      attributes: half, newton on',
 '      pair build: half/bin/atomonly/newton',
 '      stencil: half/bin/3d/newton',
 '      bin: standard',
 'Setting up Verlet run ...',
 '  Unit style    : lj',
 '  Current step  : 0',
 '  Time step     : 0.001',
 'Per MPI rank memory allocation (min/avg/max) = 6.525 | 6.525 | 6.525 Mbytes',
 'Step KinEng PotEng TotEng c_p2 c_1_temp ',
 '       0    1.7666667            0    1.7666667   0.01090535   0.59439252 ',
 '    1000    3.1462962   0.17392649    3.3202227   0.02361912    1.1654694 ',
 '    2000    2.9311648   0.13836102    3.0695258  0.021748224    1.1950624 ',
 '    3000     3.090491   0.

In [7]:
L.undump(2)
L.video("mov.mp4")